In [ ]:
!pip install pygsp

In [ ]:
from sklearn.covariance import GraphicalLasso as glasso
import numpy as np
import timeit
from scipy.io import loadmat
from scipy.io import savemat
import shlex
import numpy as np 
import matplotlib.pyplot as plt 
from time import time 
import networkx as nx
import pygsp
from sklearn import preprocessing
from scipy.optimize import linprog
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigs
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import normalized_mutual_info_score as nmi
import cvxpy as cp
from scipy.optimize import bisect
from scipy import sparse
from scipy.spatial.distance import cosine
import pickle
def save_variable(variable,filename):
  pickle.dump(variable,open(filename, "wb"))
def load_variable(filename):
  return pickle.load(open(filename,'rb')) 
def NormalizeData(data):
  return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
def Updatec(X,c_init,max_iter,rho_c,elogD,abstheta,m):
  c = c_init
  N = len(X[:,0])
  D = len(X[0,:])
  obj = []
  K = np.zeros((N,N))
  c.shape = ((N,1))
  A1 = (abstheta@np.ones((N,1)))+X@X.T@np.ones((N,1))
  for j in range(max_iter):
    K = -np.kron(c,np.ones((1,N)))-(np.kron(c,np.ones((1,N)))).T+elogD
    c = c + rho_c*(-np.linalg.inv(K+0.0001*np.eye(N))@np.ones((N,1))+A1)

    def f(x):
      return sum([min(max(c[k]-x,0),1) for k in range(N)])-m   
    lamda =  bisect(f,-100000,100000) 
    
    for i in        range(N):
      c[i] = min(max(c[i]-lamda,0),1)

    K = -np.kron(c,np.ones((1,N)))-(np.kron(c,np.ones((1,N)))).T+elogD
    obj.append(sum(abs(A0)@c)+np.log(np.linalg.det(K)+0.001)-np.trace(X@X.T@K))
    print(j,c)
    if j>1:
      if (abs(obj[j]-obj[j-1])<1.0e-4): 
        print('c Converged')
        break  

  return obj,c


In [ ]:
#Loading data - C.elegans
data = loadmat('/Datasets/Celegans.mat')
A0=np.array(data['A0'].todense())
N=len(A0[:,0])
coords = data['coords']
X = data['coords']
logD = np.zeros((N,N))
for i in range(N):
  for j in range(N):
    logD[i,j] = 0.09*np.log(1000*(np.linalg.norm(coords[i,0:2]-coords[j,0:2])**2+0.001))
elogD = 0.09*logD    

'''
To load the other datasets used in the paper:

#London underground
data = loadmat('/Datasets/London_underground.mat')
A0=np.array(data['A0'])
N=len(A0[:,0])
coords = data['Coords']
X = data['Coords']
logD = np.zeros((N,N))
for i in range(N):
  for j in range(N):
    logD[i,j] = 0.09*np.log(1000*(np.linalg.norm(coords[i,0:2]-coords[j,0:2])**2+0.001))    
elogD = 0.09*logD     

# Olympics
data = loadmat('/Datasets/olympics.mat')
X = data['X2']
A0 = data['X3']+data['X4']
N=len(A0[:,0])
elogD = np.zeros((N,N))

# Synthetic data
A0 = load_variable('/Datasets/syn/theta_syn')
N = np.size(A0,0)
X = load_variable('/Datasets/syn/X_ga_nonlinear')
D = np.size(X,1)
elogD = load_variable('/Datasets/syn/elogD_syn')    
c_true = load_variable('/Datasets/syn/c_syn')   
'''

In [ ]:
N = np.size(A0,0)
D = np.size(X,1)
obj_fn = []
c = abs(np.random.normal(0,1,(N,)))
Z = c@c.T-elogD
#Update c---------------------------------------------------------------------
obj_fn,c = Updatec(X,c,10,1e-2,elogD,abs(A0),N/8) #X,Q,c,beta,max_iter,rho_w,w_init,abstheta,m
print("c=",c)

KeyboardInterrupt: ignored